In [2]:
import re
import nltk
import time
import pandas as pd
import numpy as np
import sklearn
from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pattern.en import singularize
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
stop = stopwords.words('english')

In [3]:
#Import Data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
attributes=pd.read_csv('attributes.csv')

In [3]:
#Remove Nan
attributes=attributes.dropna(how='all')
attributes.reset_index(inplace=True)

In [4]:
#Get a stats of attributes names
all_names=attributes.groupby('name').count().sort_values('index',ascending=False)

In [5]:
attributes.product_uid.unique().__len__()

86263

In [6]:
#Attributes with al least 5% occurance
all_names.drop(all_names[all_names.value<4313].index)

,index,product_uid,value
name,,,
MFG Brand Name,86250,86250,86220
Bullet02,86248,86248,86248
Bullet03,86226,86226,86226
Bullet04,86174,86174,86174
Bullet01,85940,85940,85940
Product Width (in.),61137,61137,61137
Bullet05,60529,60529,60528
Product Height (in.),54698,54698,54698
Product Depth (in.),53652,53652,53652


In [7]:
attributes_2=attributes.copy()

In [8]:
def add_string(df):
    return reduce((lambda x, y: str(x)+' '+str(y)),df.tolist())
Color_Finish=attributes_2.loc[map(lambda x:re.search('(olor)|(inish)',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Color_Finish.columns=['Color_Finish']
Bullets=attributes_2.loc[map(lambda x:re.search('(Bullet)',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Bullets.columns=['Bullets']
Materials=attributes_2.loc[map(lambda x:re.search('(M|m)aterial',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Materials.columns=['Materials']
Types=attributes_2.loc[map(lambda x:re.search('Product Type',x)!=None,attributes_2['name'])].groupby('product_uid').agg({'value':[add_string]})
Types.columns=['Types']

In [9]:
Attr_1=pd.merge(pd.merge(pd.merge(Color_Finish,Bullets,left_index=True,right_index=True),Materials,left_index=True,right_index=True),Types,left_index=True,right_index=True)

In [10]:
Attr_2=pd.merge(Attr_1,test.groupby('product_uid').agg({'product_title':'first'}),left_index=True,right_index=True)

In [11]:
Attr_2.shape

(17191, 5)

In [12]:
Attr_1.shape

(20680, 4)

In [13]:
Attr_2.columns=['Color_Finish','Bullets','Materials','Types','Title']

In [14]:
Attr_2.head()

,Color_Finish,Bullets,Materials,Types,Title
product_uid,,,,,
100012,Tan Caramel Browns/Tans Fruitwood,96 in. W x 72 in. L Actual Blind Width is 96 i...,Bamboo,Window Treatment Natural/Woven Roman Blind/Shade,Hampton Bay Caramel Simple Weave Bamboo Rollup...
100016,Blacks,Vented canopy Built in lattice work is great f...,Steel,Gazebo,Sunjoy Calais 8 ft. x 5 ft. x 8 ft. Steel Tile...
100032,Gray Gray,"Versatile enough for the garage, basement, lau...",PVC,Side and Wall Panels,NorskWall Slatwall Panel Kit (4-Piece)
100036,White White,No installation tools required; bracket attach...,Metal,Brackets,Rubbermaid 12 in. D Single Track Bracket
100037,Black Black,3 main pockets and 6 tool pockets 4 interior e...,Fabric,Pouch Hand Tool,Husky 9-Pocket Maintenance Pouch


In [15]:
stop=stopwords.words('english')
def cleaning_text(sentence):
    if isinstance(sentence,basestring):
        sentence = unicode(sentence,'utf-8', errors='replace')
        sentence=sentence.lower()
        sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
        sentence=re.sub('_', ' ', sentence) #removes punctuations
        sentence=re.sub('\d+',' ', sentence) #removes digits
        cleaned=' '.join([w for w in sentence.split() if not w in stop]) 
        cleaned=' '.join([singularize(w) for w in sentence.split() ]) #remove pural
        #removes english stopwords
        cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
        #selecting only nouns and adjectives

        cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) 
        #removes single lettered words and digits
        cleaned=cleaned.strip()
        return cleaned
    else:
        return ''

In [18]:
Bullets_cleaned=Bullets['Bullets'].apply(lambda x: cleaning_text(x))

In [19]:
Search_Term=pd.Series(train.search_term.append(test.search_term))
Search_Term.index=train['product_uid']
Search_Term=Search_Term.apply(lambda x: cleaning_text(x))

In [17]:
tf_vectorizer1 = CountVectorizer(max_df=0.95, min_df=2, max_features=200,
                                stop_words='english')
tf_vectorizer2 = CountVectorizer(max_df=0.95, min_df=2, max_features=100,
                                stop_words='english')

In [21]:
tf1 = tf_vectorizer1.fit_transform(np.array(Bullets_cleaned.replace(np.nan, '')))
tf2 = tf_vectorizer2.fit_transform(np.array(Search_Term.replace(np.nan,'')))
TF = TfidfTransformer()

In [22]:
tf_idf_Bullets = TF.fit_transform(tf1)
tf_idf_Search_Term= TF.fit_transform(tf2)

In [23]:
Trans_Bullets=pd.DataFrame(tf_idf_Bullets)
Trans_Bullets.index=Bullets.index

In [24]:
Trans_Search_Term=pd.DataFrame(tf_idf_Search_Term)
Trans_Search_Term.index=Search_Term.index

In [25]:
Trans_Bullets.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
product_uid,,,,,,,,,,,,,,,,,,,,,
100001.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
100002.0,0.0,0.0,0.0,0.0,0.0,0.199016,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.349979,0.0,0.163457
100003.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.248274,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
100004.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
100005.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.223914,0.000000,0.0,0.270296,0.0,0.0,0.0,0.000000,0.0,0.000000


In [26]:
Trans_Search_Term.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
product_uid,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
import xgboost as xgb

/home/jian/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [29]:
train.index=train['product_uid']
Trans_Search_Term=Trans_Search_Term[0:len(train)]
Trans_Search_Term['relevance']=train['relevance']

In [30]:
train_vec=pd.merge(Trans_Bullets,Trans_Search_Term,left_index=True,right_index=True)

In [31]:
Relevance=train_vec['relevance']
train_vec=train_vec.drop(['relevance'],axis=1)

In [32]:
param={}
param['eta']=0.02
param['max_depth']=6
param['silent']=1
param['eval_metric']='mlogloss'
param['min_child_weight']=3
param['subsample']=0.7
param['colsample_bytree']=0.7
num_rounds=1000

In [38]:
start_ = time.time()

#x_train = np.array(train_vec.iloc[0:50000])
#y_train = np.array(Relevance.iloc[0:50000])

#x_validation = np.array(train_vec.iloc[50000:])
#y_validation = np.array(Relevance.iloc[50000:])
x_train, x_validation, y_train, y_validation=model_selection.train_test_split(train_vec,Relevance,test_size=0.3)

xgtrain = xgb.DMatrix(x_train, label= y_train)

clf = xgb.train(param, xgtrain, num_rounds)

xgvalidation = xgb.DMatrix(x_validation)
y_prob = clf.predict(xgvalidation)

print 'Time elapsed: %.2f seconds' % (time.time() - start_)

NameError: name 'xgb' is not defined

In [40]:
print 'MSE is ',np.std(y_prob-y_validation)

0.5197583451146538